In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.sql.types import (StructType,
                               StructField,
                               StringType,
                               IntegerType,
                               DoubleType,
                               FloatType,
                               LongType,
                               TimestampType)

In [2]:
spark = SparkSession.builder.appName("data_spark_on_emr").getOrCreate()

22/05/26 11:05:11 WARN Utils: Your hostname, OneForAll-NickdeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.6 instead (on interface en6)
22/05/26 11:05:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/26 11:05:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/26 11:05:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
spark

In [ ]:
# Stop spark session if I don't need it.
# spark.stop()

In [3]:
# spark session setting configuration
spark.sparkContext.getConf().getAll()

[('spark.sql.parquet.binaryAsString', 'true'),
 ('spark.driver.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1653534314166'),
 ('spark.sql.warehouse.dir',
  'file:/Users/oneforall_nick/workspace/Udacity_capstone_project/aws_emr_steps/spark-warehouse'),
 ('spark.executor.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.driver.host', '172.20.10.6'),
 ('spark.driver.port', '59507'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.name', 'data_spark_on_emr'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.startTime', '1653534312431'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [ ]:
# READ ALL DATA AND SAVE TO simulation data file, and then analysis those data in spark.
# ****** Read data from local file system(be a s3) to spark DataFrame ******
# All data format will be saved as a parquet file.

# file path: data >> immigration_data
# There are three tables: immigration_table, immigration_personal_table and immigration_label_table
# ***immigration_table, immigration_personal_table and immigration_label_table schema***
""""Table: immigration_main_information schema
pk: cicid -> cic_id
i94yr  -> imm_year
i94mon -> imm_month
i94citi&i94res -> imm_citi_res -> imm_cntyl
i94visa -> imm_visa
    three categories:
        1 = Business
        2 = Pleasure
        3 = Student
i94port -> imm_port
arrdate -> imm_arrival_date:
i94mode -> imm_model:
    four categories:
        1 = 'Air'
	    2 = 'Sea'
	    3 = 'Land'
	    9 = 'Not reported'
i94addr -> imm_address
    ex: 'AL'='ALABAMA'
airline -> imm_airline
fltno -> imm_flight_no
"""
# Read immigration data


"""Table: immigration_personal schema -> According to this person data that I will make a core data table to display notifications information.
pk: cicid -> cic_id
biryear -> imm_person_birth_year
gender -> imm_person_gender
visatype -> imm_person_visa_type
"""

"""Table: immigration_label schema


"""


In [ ]:
# file path: data >> news_data
"""Table: news_article schema

"""


In [1]:
# file path: data >> usCitiesDemographics_data
"""Table: us_cities_demographics schema

"""

'Table: us_cities_demographics schema\n\n'

In [4]:
data = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/immigration_data/immigration_apr16_sub.sas7bdat"

from sas7bdat import SAS7BDAT

read = SAS7BDAT(data)

read.header

Header:
	col_count_p1: 28
	col_count_p2: 0
	column_count: 28
	compression: None
	creator: None
	creator_proc: None
	date_created: 2018-05-31 13:19:46.085000
	date_modified: 2018-05-31 13:19:46.085000
	endianess: little
	file_type: DATA
	filename: immigration_apr16_sub.sas7bdat
	header_length: 65536
	lcp: 8
	lcs: 0
	mix_page_row_count: 398
	name: I94_APR16_SUB
	os_name: 
	os_type: 
	page_count: 7201
	page_length: 65536
	platform: windows
	row_count: 3096313
	row_length: 152
	sas_release: 9.0401M4
	server_type: X64_ES08R2
	u64: False

Contents of dataset "I94_APR16_SUB":
Num Name        Type   Length Format Label                                                                                
--- ----------- ------ ------ ------ -------------------------------------------------------------------------------------
  1 cicid       number      8        ID created to merge recoded ri94port variable                                        
  2 i94yr       number      8        Variable created t

In [5]:
# Get data column attributes
for i in read.columns:
    print("col_id ", i.col_id)
    print("  name",  i.name.decode(encoding ='utf-8'))
    print("  label", i.label.decode(encoding ='utf-8'))
    print("  format", i.format)
    print("  type", i.type)
    print("  length", i.length)

col_id  0
  name cicid
  label ID created to merge recoded ri94port variable
  format 
  type number
  length 8
col_id  1
  name i94yr
  label Variable created to capture the year
  format 
  type number
  length 8
col_id  2
  name i94mon
  label Variable created to capture the month
  format 
  type number
  length 8
col_id  3
  name i94cit
  label Country of Citizenship
  format 
  type number
  length 8
col_id  4
  name i94res
  label Country of Residence - i94cit if missing and all collapsed countries
  format 
  type number
  length 8
col_id  5
  name i94port
  label Port of Entry with pre-clearance recoded and all collpsed ports
  format 
  type string
  length 3
col_id  6
  name arrdate
  label Actual admission date - number
  format 
  type number
  length 8
col_id  7
  name i94mode
  label Method of transportation used to arrive in U.S.
  format 
  type number
  length 8
col_id  8
  name i94addr
  label State of intended address in U.S. used for the Summary & Analysis Tables
 

In [ ]:
# In dive data to read
# for r in read:
#     print

In [7]:
df = read.to_data_frame()
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4454822.0,2016.0,4.0,603.0,603.0,NYC,20567.0,1.0,NY,NaN,...,,,1950.0,10222016,F,,2D,9.438122e+10,00402,B2
1,4454823.0,2016.0,4.0,603.0,603.0,NYC,20567.0,1.0,NY,NaN,...,,,1952.0,10222016,F,,2D,9.437935e+10,00402,B2
2,4454824.0,2016.0,4.0,603.0,603.0,NYC,20567.0,1.0,NY,NaN,...,,,1953.0,10222016,M,,OJ,9.430155e+10,00270,B2
3,4454825.0,2016.0,4.0,603.0,603.0,NYC,20567.0,1.0,NY,NaN,...,,,1959.0,10222016,M,,2D,9.437960e+10,00402,B2
4,4454826.0,2016.0,4.0,603.0,603.0,NYC,20567.0,1.0,NY,NaN,...,,,1961.0,10222016,F,,OJ,9.430146e+10,00270,B2
